## Develop your model

In [1]:
"""Trains and Evaluates the MNIST network using a feed dictionary."""
import os

from six.moves import xrange  # pylint: disable=redefined-builtin
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.examples.tutorials.mnist import mnist


INPUT_DATA_DIR = '/tmp/tensorflow/mnist/input_data/'
MAX_STEPS = 2000
BATCH_SIZE = 100
LEARNING_RATE = 0.3
HIDDEN_1 = 128
HIDDEN_2 = 32

# HACK: Ideally we would want to have a unique subpath for each instance of the job, but since we can't
# we are instead appending HOSTNAME to the logdir
LOG_DIR = os.path.join(os.getenv('TEST_TMPDIR', '/tmp'),
                       'tensorflow/mnist/logs/fully_connected_feed/', os.getenv('HOSTNAME', ''))

class TensorflowModel():
    def train(self, **kwargs):
        tf.logging.set_verbosity(tf.logging.ERROR)
        self.data_sets = input_data.read_data_sets(INPUT_DATA_DIR)
        self.images_placeholder = tf.placeholder(
            tf.float32, shape=(BATCH_SIZE, mnist.IMAGE_PIXELS))
        self.labels_placeholder = tf.placeholder(tf.int32, shape=(BATCH_SIZE))

        logits = mnist.inference(self.images_placeholder,
                                 HIDDEN_1,
                                 HIDDEN_2)

        self.loss = mnist.loss(logits, self.labels_placeholder)
        self.train_op = mnist.training(self.loss, LEARNING_RATE)
        self.summary = tf.summary.merge_all()
        init = tf.global_variables_initializer()
        self.sess = tf.Session()
        self.summary_writer = tf.summary.FileWriter(LOG_DIR, self.sess.graph)
        self.sess.run(init)

        data_set = self.data_sets.train
        for step in xrange(MAX_STEPS):
            images_feed, labels_feed = data_set.next_batch(BATCH_SIZE, False)
            feed_dict = {
                self.images_placeholder: images_feed,
                self.labels_placeholder: labels_feed,
            }

            _, loss_value = self.sess.run([self.train_op, self.loss],
                                     feed_dict=feed_dict)
            if step % 100 == 0:
                print("At step {}, loss = {}".format(step, loss_value))
                summary_str = self.sess.run(self.summary, feed_dict=feed_dict)
                self.summary_writer.add_summary(summary_str, step)
                self.summary_writer.flush()



In [2]:
# Local Development          
remote_train = TensorflowModel()
remote_train.train()


Extracting /tmp/tensorflow/mnist/input_data/train-images-idx3-ubyte.gz
Extracting /tmp/tensorflow/mnist/input_data/train-labels-idx1-ubyte.gz
Extracting /tmp/tensorflow/mnist/input_data/t10k-images-idx3-ubyte.gz
Extracting /tmp/tensorflow/mnist/input_data/t10k-labels-idx1-ubyte.gz
At step 0, loss = 2.2995493412017822
At step 100, loss = 0.36373603343963623
At step 200, loss = 0.25089913606643677
At step 300, loss = 0.3759041726589203
At step 400, loss = 0.1888887584209442
At step 500, loss = 0.24010291695594788
At step 600, loss = 0.10854856669902802
At step 700, loss = 0.09449024498462677
At step 800, loss = 0.18540364503860474
At step 900, loss = 0.17415398359298706
At step 1000, loss = 0.08601253479719162
At step 1100, loss = 0.12495127320289612
At step 1200, loss = 0.05015036463737488
At step 1300, loss = 0.09366652369499207
At step 1400, loss = 0.12258239835500717
At step 1500, loss = 0.1578790843486786
At step 1600, loss = 0.04398500546813011
At step 1700, loss = 0.07174021750688

In [5]:
import fairing
fairing.config.set_preprocessor('python')
fairing.config.set_builder(name='append', registry='seedjeffwan',
                           base_image='tensorflow/tensorflow:1.13.1-py3')
fairing.config.run()
        

Using preprocessor: <fairing.preprocessors.base.BasePreProcessor object at 0x7f50924ac630>
Using builder: <fairing.builders.append.append.AppendBuilder object at 0x7f50924ac4a8>
Using deployer: <fairing.builders.append.append.AppendBuilder object at 0x7f50924ac4a8>
Building image using Append builder...
Creating docker context: /tmp/fairing_context_h7iyxnhh
Loading Docker credentials for repository 'tensorflow/tensorflow:1.13.1-py3'
Image successfully built in 1.0001846440136433s.
Pushing image seedjeffwan/fairing-job:BA0CB4F3...
Loading Docker credentials for repository 'seedjeffwan/fairing-job:BA0CB4F3'
Uploading seedjeffwan/fairing-job:BA0CB4F3
Layer sha256:29303e8416d52b21fcf37eef03df2d402dd4c772860dd67890862ebe3389e1c8 exists, skipping
Layer sha256:fe293195091d6aba06481f8c65f61eb206d1aaadfc6028a432194dd49e237d1c exists, skipping
Layer sha256:0a5fb8dc4fa0b9c02e5e64782ebf3288cc2a8b795e9d92ff722b06a425b756de exists, skipping
Layer sha256:9c461696bc0925124c5fe76298c2590a4d4b4009eab16a

V2DiagnosticException: response: {'content-type': 'application/json; charset=utf-8', 'docker-distribution-api-version': 'registry/2.0', 'www-authenticate': 'Bearer realm="https://auth.docker.io/token",service="registry.docker.io",scope="repository:seedjeffwan/fairing-job:pull,push",error="insufficient_scope"', 'date': 'Fri, 09 Aug 2019 00:04:05 GMT', 'content-length': '248', 'strict-transport-security': 'max-age=31536000', 'status': '401'}
authentication required: [{'Type': 'repository', 'Class': '', 'Name': 'seedjeffwan/fairing-job', 'Action': 'pull'}, {'Type': 'repository', 'Class': '', 'Name': 'seedjeffwan/fairing-job', 'Action': 'push'}]